In [1]:
import pandas as pd

In [2]:
#read csv
passenger = pd.read_csv('./SAS_training_data/training_pax_detail.csv')
baggage = pd.read_csv('./SAS_training_data/training_baggage_info.csv')
flight = pd.read_csv('./SAS_training_data/training_flt_schedule.csv')
segment = pd.read_csv('./SAS_training_data/training_od_segment.csv')
flight_data = pd.read_csv('./SAS_training_data/flt_data.csv')

In [3]:
#get info
passenger_lite = passenger[['Pax_Key', 'Pax_Prm_Id', 'Flt_Id', 'Pax_Typ_Cd', 'Gndr_Cd', 'Obd_Cbn_Cls_Cd']]
baggage_lite = baggage[['Pax_key', 'Bag_Cnt', 'Bag_Wt_Msr']]
flight_lite = flight[['Flt_Id', 'Dep_Season', 'Dep_Apt_Cd', 'Arr_Apt_Cd', 'Acft_Typ_Cd']]

In [4]:
#classify passenger dataset1
passenger_forcount = passenger[['Pax_Key', 'Flt_Id', 'Pax_Typ_Cd', 'Gndr_Cd']]
passenger_count = pd.pivot_table(passenger_forcount, index = ['Flt_Id'], columns = ['Pax_Typ_Cd', 'Gndr_Cd'], aggfunc = 'count', fill_value = 0)
passenger_count.columns = passenger_count.columns.droplevel(0)
passenger_count.columns = [' '.join(col).strip() for col in passenger_count.columns.values]

In [5]:
#classify passenger dataset2
passenger_forcount_2 = passenger[['Pax_Key', 'Flt_Id', 'Pax_Typ_Cd', 'Gndr_Cd', 'Obd_Cbn_Cls_Cd']]
passenger_count_2 = pd.pivot_table(passenger_forcount_2, index = ['Flt_Id'], columns = ['Pax_Typ_Cd', 'Gndr_Cd', 'Obd_Cbn_Cls_Cd'], aggfunc = 'count', fill_value = 0)
passenger_count_2.columns = passenger_count_2.columns.droplevel(0)
passenger_count_2.columns = [' '.join(col).strip() for col in passenger_count_2.columns.values]

In [6]:
#merge passenger and baggage
passenger_baggage = pd.merge(left = passenger_lite, right = baggage_lite, how = 'left', left_on = 'Pax_Key', right_on = 'Pax_key', indicator = False)

In [7]:
#total baggage of each flight
baggage_total = passenger_baggage.groupby(['Flt_Id'])['Bag_Wt_Msr'].sum()

In [8]:
#final dataset1
flight_baggage = pd.merge(left = flight_lite, right = baggage_total, how = 'left', left_on = 'Flt_Id', right_on = 'Flt_Id', indicator = False)
flight_baggage_passenger = pd.merge(left = flight_baggage, right = passenger_count, how = 'left', left_on = 'Flt_Id', right_on = 'Flt_Id', indicator = False)
flight_baggage_passenger.drop(['* *', 'Adult *', 'Child *', 'Infant *'], inplace = True, axis = 1)

flight_baggage_passenger = flight_baggage_passenger.replace('寬體客機', 0)
flight_baggage_passenger = flight_baggage_passenger.replace('窄體客機', 1)
flight_baggage_passenger = flight_baggage_passenger.replace('S', 0)
flight_baggage_passenger = flight_baggage_passenger.replace('W', 1)

In [9]:
#final data dataset2
flight_baggage_2 = pd.merge(left = flight_lite, right = baggage_total, how = 'left', left_on = 'Flt_Id', right_on = 'Flt_Id', indicator = False)
flight_baggage_passenger_2 = pd.merge(left = flight_baggage_2, right = passenger_count_2, how = 'left', left_on = 'Flt_Id', right_on = 'Flt_Id', indicator = False)
flight_baggage_passenger_2.drop(['* * Y','* * K','* * C', 'Adult * C','Adult * K','Adult * Y','Child * K','Child * Y', 'Infant * C','Infant * K','Infant * Y'], inplace = True, axis = 1)

flight_baggage_passenger_2 = flight_baggage_passenger_2.replace('寬體客機', 0)
flight_baggage_passenger_2 = flight_baggage_passenger_2.replace('窄體客機', 1)
flight_baggage_passenger_2 = flight_baggage_passenger_2.replace('S', 0)
flight_baggage_passenger_2 = flight_baggage_passenger_2.replace('W', 1)

In [10]:
#all stations
Dep_All = flight_baggage_passenger['Dep_Apt_Cd']
Arr_All = flight_baggage_passenger['Arr_Apt_Cd']

#distinct station
Dep_Idx = Dep_All.drop_duplicates(keep='first', inplace=False)
Arr_Idx = Arr_All.drop_duplicates(keep='first', inplace=False)
merge_idx = pd.concat([Dep_Idx,Arr_Idx], join='inner')
merge_idxs = merge_idx.drop_duplicates(keep='first', inplace=False)

#replace station with number
for idx, val in enumerate(merge_idxs) :
    Dep_All = Dep_All.replace(val, str(idx))
    Arr_All = Arr_All.replace(val, str(idx))

In [11]:
flight_baggage_passenger.drop(['Dep_Apt_Cd', 'Arr_Apt_Cd'], inplace = True, axis = 1)
flight_baggage_passenger = pd.concat([flight_baggage_passenger, Dep_All, Arr_All], axis = 1)
flight_baggage_passenger

,Flt_Id,Dep_Season,Acft_Typ_Cd,Bag_Wt_Msr,Adult F,Adult M,Child F,Child M,Infant F,Infant M,Dep_Apt_Cd,Arr_Apt_Cd
0,X_S_W1_Fri_BR5602_TPEPNH,0,0,13370.0,169,123,24,19,3,0,0,1
1,X_S_W1_Fri_BR5603_PNHTPE,0,0,7912.0,121,144,15,14,1,2,1,0
2,X_S_W1_Fri_BR5612_YNZTPE,0,1,1963.0,62,110,6,7,1,0,2,0
3,X_S_W1_Fri_BR5613_TPEYNZ,0,1,2182.0,89,66,14,11,1,0,0,2
4,X_S_W1_Fri_BR5616_YNZTPE,0,0,4477.0,89,275,3,3,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5360,X_W_W8_Wed_BR5751_PNHTPE,1,0,9120.0,131,142,5,5,1,0,1,0
5361,X_W_W8_Wed_BR5772_IAHTPE,1,0,7809.0,132,95,4,12,3,0,10,0
5362,X_W_W8_Wed_BR5773_TPEIAH,1,0,6283.0,130,118,3,8,0,0,0,10
5363,X_W_W8_Wed_BR5808_TPEMXP,1,0,4720.0,143,127,5,9,0,0,0,11


In [12]:
#dataset1
flight_baggage_passenger.to_csv(r'data_test.csv', index = False)

In [15]:
#dataset2
flight_baggage_passenger_2.to_csv(r'data_test_Cabin.csv', index = False)